In [1]:
import funs

import torch
import os

from torch.optim import Adam

config = funs.load_yaml('./config.yaml')

In [2]:
funs.set_seed(config.seed)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device =', device)

data_root_dirs = os.path.join(config.data_dir, config.snr)

# data building
machine_types = config.machine_types

train_dirs = [os.path.join(data_root_dirs, directory, 'train') for directory in machine_types]
val_dirs = [os.path.join(data_root_dirs, directory, 'val') for directory in machine_types]
test_dirs = [os.path.join(data_root_dirs, directory, 'test') for directory in machine_types]

# make dataframe
print('\nCreating train dataframe...\n' + '-' * 40)
train_df = funs.make_dataframe(config, train_dirs)
print('\nCreating val dataframe...\n' + '-' * 40)
val_df = funs.make_dataframe(config, val_dirs)
print('\nCreating test dataframe...\n' + '-' * 40)
test_df = funs.make_dataframe(config, test_dirs)

device = cuda

Creating train dataframe...
----------------------------------------


valve: 100%|██████████| 3212/3212 [00:13<00:00, 232.12it/s]



Creating val dataframe...
----------------------------------------


valve: 100%|██████████| 478/478 [00:01<00:00, 239.24it/s]



Creating test dataframe...
----------------------------------------


valve: 100%|██████████| 480/480 [00:02<00:00, 236.88it/s]


In [3]:
train_data, train_label = funs.get_data_label_arrays(train_df, config.sample_size, config.overlap)
val_data, val_label= funs.get_data_label_arrays(val_df, config.sample_size, config.overlap)
test_data, test_label  = funs.get_data_label_arrays(test_df, config.sample_size, config.overlap)

In [4]:
train_dataset = funs.MIMIIDataset(train_data, train_label)
val_dataset = funs.MIMIIDataset(val_data, val_label)
test_dataset = funs.MIMIIDataset(test_data, test_label)

In [5]:
train_loader = funs.get_dataloader(train_dataset, config.batch_size, shuffle = True)
val_loader = funs.get_dataloader(val_dataset, config.batch_size, shuffle = False)
test_loader = funs.get_dataloader(test_dataset, config.batch_size, shuffle = False)

In [6]:
model = funs.VAE(input_dim=config.sample_size, latent_dim=32).to(device)
optimizer = Adam(model.parameters(), lr = float(config.learning_rate))
loss = funs.Loss()

In [7]:
trainer = funs.Trainer(model, loss, optimizer, device)
train_loss_list = trainer.train(config.epoch, train_loader)
trainer.save(config.model_root, model_name='vae')

model_path = f'{config.model_root}/vae.pt'
trainer.model.load_state_dict(torch.load(model_path, weights_only=True))

eval_loss_list = trainer.eval(val_loader)
test_loss_list = trainer.eval(test_loader)


Starting Training... 
----------------------------------------
[EPOCH: 0] 
Train Loss: 19.52836

[EPOCH: 1] 
Train Loss: 24.84220


Starting Evaluation... 
----------------------------------------
Validation Loss: 23.19044

Starting Evaluation... 
----------------------------------------
Validation Loss: 21.21580
